In [ ]:
import sys

sys.path.append("/Users/PRVATE/Documents/tf_transformers/src/")

In [ ]:
import tensorflow as tf
import json
import time
import tensorflow_hub as hub
from tf_transformers.tokenizer import GPT2Tokenizer
from tf_transformers.models import GPT2Encoder
from tf_transformers.data import pad_dataset

from tf_transformers.text import TextDecoder

from pprint import pprint
import os

In [ ]:
# This tokenizer is not necessary
# We can use amazing HuggingFace tokenizer library also

# GPT2 Tokenizer
tokenizer_location = "/Users/PRVATE/HUggingFace_Models/tokenizer_vocab_models/"

merges_file_path = os.path.join(tokenizer_location, "gpt2/merges.txt")
vocab_path = os.path.join(tokenizer_location, "gpt2/vocab.json")

tokenizer = GPT2Tokenizer(
    merges_file_path=merges_file_path, vocab_path=vocab_path, do_lower_case=False
)

In [ ]:
# Load saved model
module_output_path = "model_pb/"
loaded = tf.saved_model.load(module_output_path)
gpt2_model_pb = loaded.signatures["serving_default"]


# Load saved model using Hub kerasLyaer
module_output_path = "model_pb/"
gpt2_model_hub = hub.KerasLayer(
    module_output_path,
    signature="serving_default",
    trainable=False,
    signature_outputs_as_dict=True,
)

In [ ]:
# Convert text to tokens (for GPT2 Model)
@pad_dataset
def tokenizer_fn(tokenizer, text_list):
    """Tokenizer fn should return a dict (no padding is required).
    Make sure, you pass all primary keys required to the model

    text_list: a list of text

    {'input_ids': tf.constant([[1, 2]]),
     'input_mask': tf.constant([[1, 1]]),
     'input_type_ids': tf.constant([[1, 0]])}


    """
    input_ids = []
    input_mask = []
    input_type_ids = []
    for text in text_list:
        input_ids.append(tokenizer.encode(text))
    inputs = {"input_ids": input_ids}
    return inputs


# # Decoder using saved_model

decoder = TextDecoder(
    model=gpt2_model_pb, hidden_dimension=768, num_attention_heads=12, num_layers=12
)

# # Decoder using hub

decoder_hub = TextDecoder(
    model=gpt2_model_hub, hidden_dimension=768, num_attention_heads=12, num_layers=12
)

In [ ]:
text_list = ["Sachin Tendulkar is one of the finest", "I like to walk with my dog"]

In [ ]:
# Greedy
start_time = time.time()
inputs = tokenizer_fn(tokenizer, text_list)
result = decoder.decode(inputs, max_iterations=25, mode="greedy")
end_time = time.time()
print("Time taken {} seconds".format(end_time - start_time))

print("_______________________________________________________")
print(result)

In [ ]:
# Greedy with EOS_ID

start_time = time.time()
inputs = tokenizer_fn(tokenizer, text_list)
result = decoder.decode(inputs, max_iterations=25, mode="greedy", eos_id=13)
end_time = time.time()
print("Time taken {} seconds".format(end_time - start_time))
print("_______________________________________________________")
print(result)

In [ ]:
# Greedy using Hub

start_time = time.time()
inputs = tokenizer_fn(tokenizer, text_list)
result = decoder_hub.decode(inputs, max_iterations=25, mode="greedy")
end_time = time.time()
print("Time taken {} seconds".format(end_time - start_time))
print("_______________________________________________________")
print(result)

In [ ]:
# Greedy using Hub EOS ID

start_time = time.time()
inputs = tokenizer_fn(tokenizer, text_list)
result = decoder_hub.decode(inputs, max_iterations=25, mode="greedy", eos_id=13)
end_time = time.time()
print("Time taken {} seconds".format(end_time - start_time))
print("_______________________________________________________")
print(result)

In [ ]:
# Beam Search

start_time = time.time()
inputs = tokenizer_fn(tokenizer, text_list)
result = decoder.decode(
    inputs, max_iterations=25, beam_size=5, mode="beam", do_sample=False, eos_id=-100
)
end_time = time.time()
print("Time taken {} seconds".format(end_time - start_time))
print("_______________________________________________________")
pprint(result)

In [ ]:
# Beam Search EOS ID

start_time = time.time()
inputs = tokenizer_fn(tokenizer, text_list)
result = decoder.decode(
    inputs, max_iterations=25, beam_size=5, mode="beam", do_sample=False, eos_id=13
)
end_time = time.time()
print("Time taken {} seconds".format(end_time - start_time))
print("_______________________________________________________")
pprint(result)

In [ ]:
# Beam Search using Hub

start_time = time.time()
inputs = tokenizer_fn(tokenizer, text_list)
result = decoder_hub.decode(
    inputs, max_iterations=25, beam_size=5, mode="beam", do_sample=False, eos_id=-100
)
end_time = time.time()
print("Time taken {} seconds".format(end_time - start_time))
print("_______________________________________________________")
pprint(result)

In [ ]:
# Beam Search Hub with EOS ID

start_time = time.time()
inputs = tokenizer_fn(tokenizer, text_list)
result = decoder_hub.decode(
    inputs, max_iterations=25, beam_size=5, mode="beam", do_sample=False, eos_id=13
)
end_time = time.time()
print("Time taken {} seconds".format(end_time - start_time))
print("_______________________________________________________")
pprint(result)

In [ ]:
# Top K Sampling , with Top P ( Nucleus )

start_time = time.time()
inputs = tokenizer_fn(tokenizer, text_list)
result = decoder.decode(
    inputs,
    max_iterations=25,
    do_sample=True,
    top_k=10,
    top_p=0.75,
    num_return_sequences=3,
    mode="top_k_top_p",
)
end_time = time.time()
print("Time taken {} seconds".format(end_time - start_time))
print("_______________________________________________________")
pprint(result)

In [ ]:
!rm -rf model_ckpt/
!rm -rf model_pb/